https://www.kaggle.com/code/krepkiioreshek/spam-classification-nlp-different-architectures

#  Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Loading Dataset

In [2]:
data = pd.read_csv('SPAM_SMS.csv')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
texts = data['Message'].values
labels = data['Category'].values

labels

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

# Turning words to tokens using Tokenizer

In [5]:
from keras.preprocessing.text import Tokenizer

In [6]:
tokenizer = Tokenizer(num_words=None,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=' ',
                      char_level=False,
                      oov_token='unknown')
tokenizer.fit_on_texts(texts)

In [7]:
X = tokenizer.texts_to_matrix(texts)
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Encoding Labels

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(labels)

Y = le.fit_transform(labels)
Y

array([0, 0, 1, ..., 0, 0, 0])

#### Split Data into Train and Test

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Split data on train and test for embedding

In [12]:
sequences = tokenizer.texts_to_sequences(texts)
sequences = np.array(sequences)

X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(sequences, Y, test_size=0.2)

#### Fully- Connected NN

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

In [14]:
# Dense

model = Sequential()

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(BatchNormalization(input_dim=60000))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

